In [1]:
# data_processor.py
import pandas as pd
import numpy as np

# --- Configuration ---
# Ensure this matches your downloaded file name from Kaggle
INPUT_CSV_PATH = 'investments_VC.csv'
OUTPUT_CLEANED_CSV_PATH = 'cleaned_investments.csv'

# --- Load Data ---
def load_data(file_path):
    """
    Loads the raw dataset from a CSV file.
    Handles potential encoding issues common with this dataset.
    """
    print(f"Loading data from {file_path}...")
    try:
        # The dataset is known to sometimes have 'latin1' encoding issues.
        # If 'utf-8' fails, 'latin1' or 'iso-8859-1' are good alternatives to try.
        df = pd.read_csv(file_path, encoding='latin1')
        print("Data loaded successfully.")
        return df
    except FileNotFoundError:
        print(f"ERROR: File not found at {file_path}. Please ensure the dataset is in the correct location and the INPUT_CSV_PATH is correct.")
        return None
    except Exception as e:
        print(f"ERROR: An error occurred while loading the data: {e}")
        return None

# --- Clean Data ---
def clean_data(df):
    """
    Cleans the input DataFrame:
    - Standardizes column names.
    - Cleans and converts 'funding_total_usd'.
    - Cleans 'country_code' (renames to 'country') and handles missing values.
    - Extracts 'primary_category' from 'category_list'.
    - Converts date columns to datetime objects and extracts years.
    - Cleans 'status' and 'funding_rounds'.
    - Drops rows with critical missing data for analysis (e.g., funding_total_usd, first_funding_year).
    """
    if df is None:
        print("No DataFrame provided to clean_data function. Skipping.")
        return None
    print("Starting data cleaning...")
    df_cleaned = df.copy()

    # 1. Standardize column names
    # Remove leading/trailing spaces, convert to lowercase, replace spaces with underscores.
    df_cleaned.columns = df_cleaned.columns.str.strip().str.lower().str.replace(' ', '_')
    print("Standardized column names.")

    # 2. Handle 'funding_total_usd'
    # This is a critical column for almost all analyses.
    if 'funding_total_usd' in df_cleaned.columns:
        print("Cleaning 'funding_total_usd'...")
        # Original data has ' - ' for missing and commas as thousands separators.
        df_cleaned['funding_total_usd'] = df_cleaned['funding_total_usd'].astype(str).str.replace(',', '')
        df_cleaned['funding_total_usd'] = df_cleaned['funding_total_usd'].replace(' - ', np.nan) # Ensure '-' is treated as NaN
        df_cleaned['funding_total_usd'] = pd.to_numeric(df_cleaned['funding_total_usd'], errors='coerce')

        initial_rows = len(df_cleaned)
        df_cleaned.dropna(subset=['funding_total_usd'], inplace=True) # Essential for funding analysis
        print(f"Dropped {initial_rows - len(df_cleaned)} rows with missing/invalid 'funding_total_usd'.")
    else:
        print("WARNING: 'funding_total_usd' column not found after standardization. Check original column names and adjust if necessary.")
        # This is a critical failure point if the column isn't found.

    # 3. Handle 'country_code' (often used as 'country')
    if 'country_code' in df_cleaned.columns:
        print("Cleaning 'country_code' and renaming to 'country'...")
        df_cleaned.rename(columns={'country_code': 'country'}, inplace=True)
        df_cleaned['country'] = df_cleaned['country'].str.strip()
        # Drop rows where country is missing, as it's important for geographical analysis
        initial_rows_country = len(df_cleaned)
        df_cleaned.dropna(subset=['country'], inplace=True)
        print(f"Dropped {initial_rows_country - len(df_cleaned)} rows with missing 'country'.")
        print("Cleaned 'country'.")
    else:
        print("WARNING: 'country_code' column not found.")


    # 4. Handle 'category_list' (extract primary category)
    if 'category_list' in df_cleaned.columns:
        print("Extracting 'primary_category' from 'category_list'...")
        # Take the first category before the '|' delimiter.
        df_cleaned['primary_category'] = df_cleaned['category_list'].astype(str).apply(
            lambda x: x.split('|')[0].strip() if pd.notnull(x) and '|' in x else (x.strip() if pd.notnull(x) else 'Unknown')
        )
        # Handle cases where category might be empty string or 'nan' after split
        df_cleaned['primary_category'] = df_cleaned['primary_category'].replace({'nan': 'Unknown', '': 'Unknown'})
        df_cleaned.loc[df_cleaned['primary_category'].isnull(), 'primary_category'] = 'Unknown'
        print("Extracted 'primary_category'.")
    else:
        print("WARNING: 'category_list' column not found.")

    # 5. Handle Date Columns
    date_cols_to_process = ['founded_at', 'first_funding_at', 'last_funding_at']
    print(f"Cleaning date columns: {date_cols_to_process}...")
    for col in date_cols_to_process:
        if col in df_cleaned.columns:
            df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')
            # Extract year for easier filtering and aggregation in Streamlit
            # Using .dt.year will result in float if there are NaT values, then convert to Int64 to support NA
            year_col_name = col.replace('_at', '_year')
            df_cleaned[year_col_name] = df_cleaned[col].dt.year.astype('Int64') # Int64 supports pandas NA
        else:
            print(f"WARNING: Date column '{col}' not found.")
    print("Cleaned date columns and extracted years.")

    # 6. Clean 'status' column
    if 'status' in df_cleaned.columns:
        print("Cleaning 'status' column...")
        df_cleaned['status'] = df_cleaned['status'].fillna('unknown').str.lower()
        # Consolidate status values if needed (e.g., map variations to standard terms)
        # Common values are 'operating', 'acquired', 'closed', 'ipo'.
        valid_statuses = ['operating', 'acquired', 'closed', 'ipo']
        df_cleaned['status'] = df_cleaned['status'].apply(lambda x: x if x in valid_statuses else 'unknown')
        print("Cleaned 'status' column.")
    else:
        print("WARNING: 'status' column not found.")

    # 7. Clean 'funding_rounds'
    if 'funding_rounds' in df_cleaned.columns:
        print("Cleaning 'funding_rounds'...")
        df_cleaned['funding_rounds'] = pd.to_numeric(df_cleaned['funding_rounds'], errors='coerce')
        df_cleaned.dropna(subset=['funding_rounds'], inplace=True) # Drop if not a number
        df_cleaned['funding_rounds'] = df_cleaned['funding_rounds'].astype(int)
        print("Cleaned 'funding_rounds'.")
    else:
        print("WARNING: 'funding_rounds' column not found.")

    # 8. Rename 'name' if it's messy (e.g. 'name ' with space)
    # Check if a common problematic name like 'name ' (with trailing space) exists and 'name' does not
    if 'name ' in df_cleaned.columns and 'name' not in df_cleaned.columns:
        df_cleaned.rename(columns={'name ': 'name'}, inplace=True)
        print("Renamed 'name ' to 'name'.")
    elif 'name' not in df_cleaned.columns:
         print("WARNING: Standard 'name' column not found. Company identification might be affected.")


    # 9. Drop rows where 'first_funding_year' is NaN as it's critical for many time-based analyses
    if 'first_funding_year' in df_cleaned.columns:
        initial_rows_ffy = len(df_cleaned)
        df_cleaned.dropna(subset=['first_funding_year'], inplace=True)
        # After dropping NaNs, we can safely convert to int if it's Int64
        df_cleaned['first_funding_year'] = df_cleaned['first_funding_year'].astype(int)
        print(f"Dropped {initial_rows_ffy - len(df_cleaned)} rows with missing 'first_funding_year'.")
    else:
        print("WARNING: 'first_funding_year' column not found or not processed. Time-based analysis will be affected.")


    print(f"Data cleaning completed. Shape of cleaned data: {df_cleaned.shape}")
    print("\n--- Missing Values Summary (Top 10) After Cleaning ---")
    # Display columns with the most missing values after cleaning
    missing_summary = df_cleaned.isnull().sum()
    missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
    print(missing_summary.head(10))
    return df_cleaned

# --- Perform EDA (Conceptual - details in Analysis Report) ---
def perform_basic_eda(df):
    """
    Performs and prints basic Exploratory Data Analysis results.
    This is a lightweight EDA; more detailed EDA should be in the Analysis Report.
    """
    if df is None or df.empty:
        print("Skipping EDA as data is not available or empty.")
        return

    print("\n--- Basic Exploratory Data Analysis (EDA) ---")

    if 'funding_total_usd' in df.columns:
        print("\n--- Descriptive Statistics for 'funding_total_usd' ---")
        print(df['funding_total_usd'].describe())
    else:
        print("'funding_total_usd' not found for EDA.")

    if 'primary_category' in df.columns:
        print("\n--- Value Counts for Top 5 Primary Categories ---")
        print(df['primary_category'].value_counts().nlargest(5))
    else:
        print("'primary_category' not found for EDA.")

    if 'country' in df.columns:
        print("\n--- Value Counts for Top 5 Countries ---")
        print(df['country'].value_counts().nlargest(5))
    else:
        print("'country' not found for EDA.")

    if 'status' in df.columns:
        print("\n--- Value Counts for Company Status ---")
        print(df['status'].value_counts())
    else:
        print("'status' not found for EDA.")

    # Check availability of data for potential KPIs (to be visualized in Streamlit)
    print("\n--- Data Check for Potential KPIs ---")
    kpi_columns_to_check = {
        "Total Funding by Year": ['first_funding_year', 'funding_total_usd'],
        "Top N Countries by Funding": ['country', 'funding_total_usd'],
        "Top N Categories by Funding": ['primary_category', 'funding_total_usd'],
        "Funding by Company Status": ['status', 'funding_total_usd'],
        "Average Funding per Round": ['funding_total_usd', 'funding_rounds'],
        "Number of Investments Over Time": ['first_funding_year', 'name'], # Using 'name' as a proxy for unique investment
        "Distribution of Funding Amounts": ['funding_total_usd'],
        "Funding Rounds vs. Total Funding": ['funding_rounds', 'funding_total_usd'],
        "Top Cities by Investment (if 'city' available)": ['city', 'funding_total_usd'],
        "Time from Founding to First Funding": ['founded_year', 'first_funding_year']
    }
    for kpi, cols in kpi_columns_to_check.items():
        missing_cols = [col for col in cols if col not in df.columns or df[col].isnull().all()]
        if not missing_cols:
            print(f"Data for KPI '{kpi}': Seems available.")
        else:
            print(f"Data for KPI '{kpi}': MISSING or all NaNs for columns: {missing_cols}")


# --- Main Execution Block ---
if __name__ == "__main__":
    print("--- Starting Startup Investment Data Processing ---")
    raw_df = load_data(INPUT_CSV_PATH)

    if raw_df is not None:
        print("\n--- Initial Data Info (First 5 Rows & Column Info) ---")
        print("Head of raw data:")
        print(raw_df.head())
        print("\nInfo of raw data:")
        raw_df.info(verbose=True, show_counts=True) # More detailed info

        cleaned_df = clean_data(raw_df)

        if cleaned_df is not None and not cleaned_df.empty:
            perform_basic_eda(cleaned_df)
            # Save the cleaned data to a new CSV file
            try:
                cleaned_df.to_csv(OUTPUT_CLEANED_CSV_PATH, index=False)
                print(f"\nCleaned data successfully saved to '{OUTPUT_CLEANED_CSV_PATH}'")
            except Exception as e:
                print(f"ERROR: Could not save cleaned data: {e}")
        else:
            print("Cleaned data is empty or None. Not performing EDA or saving.")
    else:
        print("Raw data could not be loaded. Processing halted.")
    print("--- End of Startup Investment Data Processing ---")


--- Starting Startup Investment Data Processing ---
Loading data from investments_VC.csv...
Data loaded successfully.

--- Initial Data Info (First 5 Rows & Column Info) ---
Head of raw data:
                         permalink                name  \
0            /organization/waywire            #waywire   
1  /organization/tv-communications  &TV Communications   
2    /organization/rock-your-paper   'Rock' Your Paper   
3   /organization/in-touch-network   (In)Touch Network   
4   /organization/r-ranch-and-mine  -R- Ranch and Mine   

                    homepage_url  \
0         http://www.waywire.com   
1          http://enjoyandtv.com   
2   http://www.rockyourpaper.org   
3  http://www.InTouchNetwork.com   
4                            NaN   

                                       category_list        market   \
0         |Entertainment|Politics|Social Media|News|          News    
1                                            |Games|         Games    
2                            